# Take Two

In [1]:
import numpy as np
from datasets import Dataset
import os
from pathlib import Path
import torch
import sys
curdir = Path(os.getcwd())
sys.path.append(str(curdir.parent.absolute()))

from src.utils.data import read_pickle


# Load test dataset
GO_ANNOTATIONS_PATH = "/home/samirchar/ProteinFunctions/data/annotations/go_annotations_2019_07_01_updated.pkl"

go_annotations = read_pickle(GO_ANNOTATIONS_PATH)
from src.utils.data import read_json
embeddings_path ="/home/samirchar/ProteinFunctions/data/embeddings/frozen_E5_multiling_inst_label_embeddings_mean.pt" 
embeddings_idx_path ="/home/samirchar/ProteinFunctions/data/embeddings/frozen_E5_multiling_inst_label_embeddings_mean_index.pt" 

embeddings = torch.load(embeddings_path)
embeddings_index = torch.load(embeddings_idx_path)

# Get some labels
text = go_annotations['label'].tolist()

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [22]:
embeddings_path ="/home/samirchar/ProteinFunctions/data/embeddings/2024_E5_multiling_inst_frozen_label_embeddings_mean.pt" 
embeddings_idx_path ="/home/samirchar/ProteinFunctions/data/embeddings/2024_E5_multiling_inst_frozen_label_embeddings_mean_index.pt" 
embeddings = torch.load(embeddings_path)
embeddings_index = torch.load(embeddings_idx_path)
text=embeddings_index[(embeddings_index['id']=='GO:0140723')&(embeddings_index['description_type']=='name')]['description'].tolist()
text_embeddings = embeddings[embeddings_index[(embeddings_index['id']=='GO:0140723')&(embeddings_index['description_type']=='name')].index[0]]
print(text)
print(text_embeddings,text_embeddings.sum())

["Instruct: Identify the main categories, themes, or topics described in the following Gene Ontology (GO) term, which is used to detail a protein's function\nQuery: patulin biosynthetic process"]
tensor([ 0.7899,  0.3719, -0.3108,  ..., -0.2164, -0.3004,  0.4498]) tensor(-8.3121)


In [23]:
torch.where(embeddings == 0.7906093597412109)

(tensor([9304]), tensor([770]))

In [26]:
embeddings[9304,770]

tensor(0.7906)

In [5]:
text

["Instruct: Identify the main categories, themes, or topics described in the following Gene Ontology (GO) term, which is used to detail a protein's function\nQuery: patulin biosynthetic process"]

In [3]:
embeddings_path ="/home/samirchar/ProteinFunctions/data/embeddings/2024_E5_multiling_inst_sos_false_frozen_label_embeddings_mean.pt" 
embeddings_idx_path ="/home/samirchar/ProteinFunctions/data/embeddings/2024_E5_multiling_inst_sos_false_frozen_label_embeddings_mean_index.pt" 
embeddings = torch.load(embeddings_path)
embeddings_index = torch.load(embeddings_idx_path)
text=embeddings_index[(embeddings_index['id']=='GO:0140723')&(embeddings_index['description_type']=='name')]['description'].tolist()
text_embeddings = embeddings[embeddings_index[(embeddings_index['id']=='GO:0140723')&(embeddings_index['description_type']=='name')].index[0]]
print(text)
print(text_embeddings,text_embeddings.sum())

["Instruct: Identify the main categories, themes, or topics described in the following Gene Ontology (GO) term, which is used to detail a protein's function\nQuery: patulin biosynthetic process"]
tensor([ 0.7931,  0.3759, -0.3119,  ..., -0.2140, -0.3011,  0.4528]) tensor(-8.3237)


In [29]:
embeddings_path ="/home/samirchar/ProteinFunctions/data/embeddings/frozen_E5_multiling_inst_label_embeddings_mean.pt" 
embeddings_idx_path ="/home/samirchar/ProteinFunctions/data/embeddings/frozen_E5_multiling_inst_label_embeddings_mean_index.pt" 
embeddings = torch.load(embeddings_path)
embeddings_index = torch.load(embeddings_idx_path)
text=embeddings_index[(embeddings_index['id']=='GO:0140723')&(embeddings_index['description_type']=='name')]['description'].tolist()
text_embeddings = embeddings[embeddings_index[(embeddings_index['id']=='GO:0140723')&(embeddings_index['description_type']=='name')].index[0]]
print(text)
print(text_embeddings,text_embeddings.sum())

["Instruct: Identify the main categories, themes, or topics described in the following Gene Ontology (GO) term, which is used to detail a protein's function\nQuery: patulin biosynthetic process"]
tensor([ 0.7906,  0.3704, -0.3123,  ..., -0.2163, -0.2990,  0.4475]) tensor(-8.3131)


In [15]:
torch.where(embeddings == 0.7906093597412109)

(tensor([ 10993, 188642]), tensor([770,   0]))

In [18]:
embeddings[188642]

tensor([ 0.7906,  0.3704, -0.3123,  ..., -0.2163, -0.2990,  0.4475])

In [122]:
text_embeddings,text_embeddings.mean()

(tensor([ 0.7899,  0.3719, -0.3108,  ..., -0.2164, -0.3004,  0.4498]),
 tensor(-0.0081))

In [72]:
from src.utils.data import read_fasta
import torch

In [98]:
from transformers import AutoTokenizer,AutoModel

checkpoint = 'intfloat/multilingual-e5-large-instruct'
model = AutoModel.from_pretrained(
    checkpoint,
    # torch_dtype=torch.float16,
).to("cuda")

# Initialize label tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

default_args = {
    "output_dir": "tmp",
    "evaluation_strategy": "no",
    "do_eval": False,
    "num_train_epochs": 1,
    "log_level": "error",
    "report_to": "none",
}

# Tokenize the go_annotations list
tokenized_data = tokenizer(text, padding="longest", truncation=True, max_length=510, return_tensors="pt")

# Create random labels for the new dataset
random_labels = np.random.randint(0, 1, (len(text)))

# Create the dataset from the tokenized data and random labels
dummy_dataset = Dataset.from_dict({
    "input_ids": tokenized_data["input_ids"],
    "attention_mask": tokenized_data["attention_mask"], 
    "labels": random_labels
})

# Set the format to PyTorch tensors
dummy_dataset.set_format("pt")

In [23]:

import re 
def apply_lora_biogpt_attention(layer,rank,alpha,device,in_features= 1024, out_features= 1024):
    # layer.self_attn.q_proj = lora.Linear(
    #     in_features, out_features, r=rank,lora_alpha=alpha)  
    # layer.self_attn.v_proj = lora.Linear(
    #     in_features, out_features, r=rank,lora_alpha=alpha)
    # layer.self_attn.k_proj = lora.Linear(
    #     in_features, out_features, r=rank,lora_alpha=alpha)
    layer.self_attn.out_proj = lora.Linear(
        in_features, out_features, r=rank,lora_alpha=alpha)
    # layer.fc1 = lora.Linear(
    #     in_features, out_features*4, r=rank,lora_alpha=alpha)
    # layer.fc2 = lora.Linear(
    #     in_features*4, out_features, r=rank,lora_alpha=alpha)
    

    layer=layer.to(device)
    
def biogpt_train_last_n_layers(model,n,lora_params=None):
    for param in model.parameters():
        param.requires_grad = False

    if n>0:
        max_layer_num = len(model.layers)-1
        for param_name,param in model.named_parameters():
            layer_num = re.search(r'layers\.(\d+)', param_name)
            if layer_num:
                number = int(layer_num.group(1))
                if number>max_layer_num-n:
                    param.requires_grad = True
                    if lora_params is not None:

                        apply_lora_biogpt_attention(**{**lora_params,
                                                     'layer':model.layers[number]}
                                                     )
        
        if lora_params is not None:
            lora.mark_only_lora_as_trainable(model)
        
        #Always train last layer norm.
        for param in model.layer_norm.parameters():
            param.requires_grad = True


In [11]:
biogpt_train_last_n_layers(model,
                           0,
                           lora_params={'rank':8,'alpha':4,'in_features':1024,'out_features':1024,'device':'cuda:0'}
                           )

for param_name,param in model.named_parameters():
    if param.requires_grad:
        print(param_name)
print_trainable_parameters(model)



trainable params: 0 || all params: 346763264 || trainable%: 0.00


In [89]:

def pool_embeddings(last_hidden_states,attention_mask,method,account_for_sos = True):
    '''
    '''
    sequence_length_raw = attention_mask.sum(dim=1, keepdim=True) #includind SOS token

    sequence_length = sequence_length_raw - 1*account_for_sos

 
    if method=='mean':
        #Account for SOS token
        adjusted_attention_mask = attention_mask.clone()
        if account_for_sos:
            adjusted_attention_mask[:,0]=0
 
        # Mask the last_hidden_state tensor and compute the sum
        sum_hidden_states = (last_hidden_states *
                                adjusted_attention_mask.unsqueeze(-1)).sum(dim=1)
 
        # Compute the mean of the last hidden state
        sequence_embedding = sum_hidden_states / (sequence_length) 
 
    elif method == 'last_token':
        last_token_indices = (sequence_length_raw - 1)\
            .unsqueeze(-1)\
            .expand(-1, -1, last_hidden_states.size(-1))
 
        sequence_embedding = last_hidden_states.gather(1, last_token_indices).squeeze()
    elif method == 'all':
 
        sequence_embedding = last_hidden_states
 
    return sequence_embedding

In [ ]:
raw_attn_scorer = torch.nn.Linear(1024,1, bias=True).to('cuda:0')

def additive_attention(hidden_states,attention_mask):
    raw_attn_scores = raw_attn_scorer(hidden_states).squeeze(-1)
    
    #Masked scored for softmax
    raw_attn_scores = raw_attn_scores.masked_fill(attention_mask==0,float('-inf'))

    #Normalized attention weights
    attn_weights = torch.softmax(raw_attn_scores,dim=-1)

    #Get final label embedding
    return torch.bmm(attn_weights.unsqueeze(1),hidden_states).squeeze(1)

In [ ]:
def _get_joint_embeddings(P_e, L_e, num_sequences,num_labels):

    sequence_embedding_dim = P_e.shape[1]
    label_embedding_dim = L_e.shape[1]

    # Use broadcasting so we don't have to expand the tensor dimensions
    joint_embeddings = torch.cat([
        P_e[:, None, :].expand(
            num_sequences, num_labels, sequence_embedding_dim),
        L_e[None, :, :].expand(
            num_sequences, num_labels, label_embedding_dim)
    ], dim=2).reshape(-1, sequence_embedding_dim + label_embedding_dim)

    return joint_embeddings

In [ ]:
from torchvision.ops import MLP


In [ ]:
P_f = torch.rand((32,1100)).to('cuda:0')

#L_e = torch.rand((32102,1024))
L_f = additive_attention(label_embeddings,masks)
W_p = MLP(1100,[1024]*1,bias=False,norm_layer=torch.nn.BatchNorm1d).to('cuda:0')
W_l = MLP(1024,[1024]*1,bias=False,norm_layer=torch.nn.BatchNorm1d).to('cuda:0')

L_e = W_l(L_f)
P_e = W_p(P_f)

In [ ]:
joint=_get_joint_embeddings(P_e.to('cuda:0'), L_e, 32,32102)

In [ ]:
joint.shape,P_e.shape,L_e.shape,P_f.shape,L_f.shape

In [ ]:
tokenized_data['input_ids']=torch.randint(0,10000,(1000,60)).to('cuda:0')
tokenized_data['attention_mask']=torch.randint(0,1,(1000,60)).to('cuda:0')

In [90]:
tokenized_data['input_ids']

tensor([[     0,    360,  36716,     12,  71257,  40383,     70,   5201,  39283,
              7,      4,   2856,     90,      4,    707,  28451,      7, 151552,
             23,     70,  25632,  46980,   2161,     18,  25443,     15,  19930,
             16,  13579,      4,   3129,     83,  11814,     47,  22443,     10,
          21308,     25,      7,  32354,  58836,     53,     12,   2340,  53311,
           3530,  12654,   2347,   9523,   9433,      2]])

In [91]:
text

["Instruct: Identify the main categories, themes, or topics described in the following Gene Ontology (GO) term, which is used to detail a protein's function\nQuery: patulin biosynthetic process"]

In [27]:
torch.cuda.memory_allocated('cuda:0')

2239570944

In [20]:
np.where(go_annotations.index=='GO:2001317')

(array([47400]),)

In [110]:
import torch.autograd.profiler as profiler
import torch
from torch.cuda.amp import autocast

bs = 1000
embeddings = []
model.eval()
for i in range(1):
    print(i)
    with autocast(), torch.set_grad_enabled(False):

        embeddings.append(
            pool_embeddings(
                model(
                    input_ids = tokenized_data['input_ids'][:bs,:].to('cuda:0'),
                    attention_mask = tokenized_data['attention_mask'][:bs,:].to('cuda:0')).last_hidden_state,
                    tokenized_data['attention_mask'][:bs,:].to('cuda:0'),
                    method='mean',
                    account_for_sos=False
                    ),
                    
                )   
    print(torch.cuda.memory_allocated('cuda:0'))

0
2248168448


In [111]:
embeddings,embeddings[0].sum()

([tensor([[ 0.7928,  0.3759, -0.3129,  ..., -0.2123, -0.3003,  0.4511]],
         device='cuda:0')],
 tensor(-8.3214, device='cuda:0'))

In [ ]:
zero_shot_embeddings

In [ ]:
zero_shot_vocabularies

In [ ]:
len(embeddings)

In [ ]:
81.2/2744

In [ ]:
len(embeddings)

In [ ]:
torch.cuda.memory_allocated('cuda:0')

In [ ]:
torch.cuda.memory_allocated('cuda:0')

In [ ]:
(13235055616-1387511808)/1e9

In [ ]:
import torch.autograd.profiler as profiler
import torch
with profiler.profile(with_stack=True, profile_memory=True) as prof:

    outs = []
    with torch.set_grad_enabled(True):
        for i in range(13):
            print(i)
            out = model(input_ids = tokenized_data['input_ids'].to('cuda:0'),attention_mask = tokenized_data['attention_mask'].to('cuda:0')).last_hidden_state
            sequence_embedding = pool_embeddings(out,tokenized_data['attention_mask'].to('cuda:0'),method='last_token')
            outs.append(sequence_embedding)

In [ ]:
print_gpu_utilization()
torch.cuda.memory_allocated('cuda:0')

In [ ]:
print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_memory_usage', row_limit=50))


In [ ]:
print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))


In [ ]:
torch.cuda.memory_allocated('cuda:0')

In [ ]:
5600617472/1606418432

In [ ]:
torch.cuda.memory_allocated('cuda:0')

In [ ]:
5600617472 - 5599797248

In [ ]:
print_gpu_utilization()

In [ ]:
sequence_embedding = pool_embeddings(out.last_hidden_state,tokenized_data['attention_mask'].to('cuda:0'),method='all')

In [ ]:
raw_attn_scorer = nn.Linear(sequence_embedding.shape[-1],1, bias=True).to('cuda:0')


In [ ]:
 
raw_attn_scores = raw_attn_scorer(sequence_embedding).squeeze(-1)
softmax_mask = torch.where(tokenized_data['attention_mask'].to('cuda:0')==0,-torch.inf,tokenized_data['attention_mask'].to('cuda:0'))
attn_weights = torch.softmax(raw_attn_scores+softmax_mask,dim=-1)
sequence_embedding_ = (sequence_embedding*attn_weights.unsqueeze(-1)).sum(dim=1)

In [ ]:
sequence_embedding_.sum()

In [ ]:
 
raw_attn_scores = raw_attn_scorer(sequence_embedding).squeeze(-1)

#Masked scored for softmax
raw_attn_scores = raw_attn_scores.masked_fill(tokenized_data['attention_mask'].to('cuda:0')==0,float('-inf'))

#softmax_mask = torch.where(tokenized_data['attention_mask'].to('cuda:0')==0,-torch.inf,tokenized_data['attention_mask'].to('cuda:0'))
attn_weights = torch.softmax(raw_attn_scores+softmax_mask,dim=-1)
sequence_embedding___ = torch.bmm(attn_weights.unsqueeze(1),sequence_embedding)

In [ ]:
(sequence_embedding_-sequence_embedding___.squeeze(1)).sum()

In [ ]:
sequence_embedding___.sum()

In [ ]:
torch.softmax(raw_attn_scores[0][:28],dim=0)

In [ ]:
raw_attn_scores

In [ ]:
torch.softmax(raw_attn_scores+softmax_mask,dim=-1)

In [ ]:
def compute_mean_hidden_states(last_hidden_states, attention_mask):
    """Compute the mean of the last hidden state for only the relevant tokens."""
    # Compute the number of relevant tokens for each sequence
    num_relevant_tokens = attention_mask.sum(dim=1, keepdim=True)
    # Mask the last_hidden_state tensor and compute the sum
    sum_hidden_states = (last_hidden_states *
                         attention_mask.unsqueeze(-1)).sum(dim=1)
    # Compute the mean of the last hidden state
    return sum_hidden_states / num_relevant_tokens

compute_mean_hidden_states(out.last_hidden_state,tokenized_data['attention_mask'].to('cuda:0'))


In [ ]:
score(out.last_hidden_state).shape

In [ ]:
linear_output = score(out.last_hidden_state)
idxs = (torch.ne(tokenized_data['input_ids'].to('cuda:0'), model.config.pad_token_id).sum(-1) - 1).to('cuda:0')
linear_output[torch.arange(2, device='cuda:0'), idxs.squeeze()]


In [ ]:
from transformers import TrainingArguments, Trainer, logging

logging.set_verbosity_error()


test_model = model

# Check if model is on cuda
print(f"Model on cuda: {next(test_model.parameters()).is_cuda}")

batch_size = 40
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size, 
    remove_unused_columns=False,
    fp16=True,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,
    **default_args
)
trainer = Trainer(model=test_model, args=training_args, train_dataset=dummy_dataset)
result = trainer.train()
print_summary(result)



train full model: memory 62150 MB

train last layer: memory 16900 MB

train last 2 layers: memory 18820  MB

train last 3 layers: memory 20620  MB

train last 4 layers: memory 22582  MB

train full model + lora: memory 54080 MB



LoRa:

Model on cuda: True
{'train_runtime': 14.3185, 'train_samples_per_second': 69.84, 'train_steps_per_second': 1.746, 'train_loss': 1.344638671875, 'epoch': 1.0}
Time: 14.32
Samples/second: 69.84
GPU memory occupied: 55019 MB.